In [1]:
%matplotlib notebook
%reset -f


import pandas as pd
import numpy as np
from scipy.stats import norm, probplot
import statsmodels.api as sm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.feature_selection import VarianceThreshold

In [2]:
def regress(X,Y):
    X_cp = sm.add_constant(X)
    model = sm.OLS(Y,X)
    results = model.fit()
    return results


def dummify(data, column_name):
    """
        Converts the column column_name in dummies / one-hot, adds them to the df and returns the df without
        original undummified columns
    """
    df = data.copy()
    df2 = pd.concat([df.drop(column_name, axis=1), pd.get_dummies(data[column_name], prefix=column_name)], axis=1)
    return df2

def dummify2(data, column_name):
    return pd.get_dummies(data[column_name], prefix=column_name)
    
def variance_threshold_selector(data, threshold):
    selector = VarianceThreshold(threshold)
    selector.fit(data)
    return data[data.columns[selector.get_support(indices=True)]]

In [9]:
#Importing excel file and formatting as a pandas dataframe
db = pd.read_excel('base.xlsx')
db.set_index("Country")
db.isnull().sum().sum()
db

,Country,VDEM,1,3,3.0 - No data,5,7.1 - Yes,"7.2 - No, but specifc limit",7.3 - No,9,...,63.3 - Suspension of public funding,63.4 - Loss of public funding,63.5 - Loss of politican rights,63.6 - Prison,63.7 - Deregistration of party,63.8 - Loss of nomination of candidate,63.9 - Loss of elected office,63.10 - Suspension of political party,63.11 - Forfeiture,63.12 - Not Applicable
0,Angola,0.386,1,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
1,Argentina,0.812,1,1,0,1,1,0,0,1,...,0,1,1,0,0,0,0,0,0,0
2,Australia,0.838,1,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
3,Belgium,0.882,0,1,0,1,0,1,0,1,...,0,1,0,0,0,0,0,0,0,0
4,Brazil,0.674,1,1,0,1,1,0,0,1,...,0,1,0,0,1,1,1,0,0,0
5,Canada,0.866,1,1,0,1,1,0,0,1,...,0,0,0,1,1,0,0,0,1,0
6,Chile,0.773,1,1,0,1,0,1,0,1,...,0,0,0,1,1,0,0,1,0,0
7,Colombia,0.667,1,0,0,0,1,0,0,1,...,0,1,0,1,1,1,1,1,1,0
8,Czech Republic,0.796,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
9,Denmark,0.900,0,0,0,0,0,0,1,0,...,0,1,0,1,0,0,0,0,0,0


In [12]:
#Excluding 'countries' and redefining columns
columns = list(db.columns)
#Correction
db = db.reindex(columns=columns)

#Organized columns to draw from
columns

['Country',
 'VDEM',
 1,
 3,
 '3.0 - No data',
 5,
 '7.1 - Yes',
 '7.2 - No, but specifc limit',
 '7.3 - No',
 9,
 '9.0 - No data',
 11,
 '11.0 - No data',
 '13.01 - Legal entities',
 '13.02 -Other',
 '13.-3 - Organizations connected to gambling',
 '13.04 - Organizations in debt',
 '13.05 - No',
 '13.06 - Religious organizations',
 '13.07 - Charities',
 '13.08 - Recently founded organizations',
 '16.0 - No data',
 '16.1 - Yes, for natural persons',
 '16.2 - Yes, for legal persons',
 '18.0 - No data',
 '18.1 - No',
 '18.2. - Yes, for natural persons',
 '18.3- Yes, for legal persons',
 '18.4 - Regular limits apply',
 23,
 '23.0 - No data',
 25,
 '25.0 - No data',
 '25.1 - Sometimes',
 26,
 '26.0 - No data',
 '26.1 - Sometimes',
 '39.0 - No data',
 '39.1 - No',
 '39.2 - Yes, funds for gender activities',
 '39.3 - Yes, other',
 2,
 '2.0 - No data',
 '2.0 - N/A',
 4,
 '4.0 - N/A',
 6,
 '6.0 - N/A',
 8,
 '8.0 - N/A',
 '8.1 - No, bit specific limit',
 10,
 '10.0 - No data',
 '10.0 - N/A',
 12

In [13]:
colunas = list(db.columns)
colunas.remove('Country')
#Redefining dataframe
df = db[colunas]
df

,VDEM,1,3,3.0 - No data,5,7.1 - Yes,"7.2 - No, but specifc limit",7.3 - No,9,9.0 - No data,...,63.3 - Suspension of public funding,63.4 - Loss of public funding,63.5 - Loss of politican rights,63.6 - Prison,63.7 - Deregistration of party,63.8 - Loss of nomination of candidate,63.9 - Loss of elected office,63.10 - Suspension of political party,63.11 - Forfeiture,63.12 - Not Applicable
0,0.386,1,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
1,0.812,1,1,0,1,1,0,0,1,0,...,0,1,1,0,0,0,0,0,0,0
2,0.838,1,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0.882,0,1,0,1,0,1,0,1,0,...,0,1,0,0,0,0,0,0,0,0
4,0.674,1,1,0,1,1,0,0,1,0,...,0,1,0,0,1,1,1,0,0,0
5,0.866,1,1,0,1,1,0,0,1,0,...,0,0,0,1,1,0,0,0,1,0
6,0.773,1,1,0,1,0,1,0,1,0,...,0,0,0,1,1,0,0,1,0,0
7,0.667,1,0,0,0,1,0,0,1,0,...,0,1,0,1,1,1,1,1,1,0
8,0.796,1,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
9,0.900,0,0,0,0,0,0,1,0,0,...,0,1,0,1,0,0,0,0,0,0


In [14]:
#Defining regression variables
Y = df['VDEM']
colunas.remove('VDEM')
X = df[colunas]
X

,1,3,3.0 - No data,5,7.1 - Yes,"7.2 - No, but specifc limit",7.3 - No,9,9.0 - No data,11,...,63.3 - Suspension of public funding,63.4 - Loss of public funding,63.5 - Loss of politican rights,63.6 - Prison,63.7 - Deregistration of party,63.8 - Loss of nomination of candidate,63.9 - Loss of elected office,63.10 - Suspension of political party,63.11 - Forfeiture,63.12 - Not Applicable
0,1,0,0,0,0,0,1,0,0,1,...,1,0,0,0,0,0,0,0,0,0
1,1,1,0,1,1,0,0,1,0,1,...,0,1,1,0,0,0,0,0,0,0
2,1,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,1,0,1,0,1,0,1,0,1,...,0,1,0,0,0,0,0,0,0,0
4,1,1,0,1,1,0,0,1,0,1,...,0,1,0,0,1,1,1,0,0,0
5,1,1,0,1,1,0,0,1,0,1,...,0,0,0,1,1,0,0,0,1,0
6,1,1,0,1,0,1,0,1,0,1,...,0,0,0,1,1,0,0,1,0,0
7,1,0,0,0,1,0,0,1,0,1,...,0,1,0,1,1,1,1,1,1,0
8,1,0,0,0,1,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,1,0,0,0,...,0,1,0,1,0,0,0,0,0,0


In [15]:
# K most relevant correlations

l = db.corr().head(1)
l = l.transpose()
l = l.sort_values(by=['VDEM'],ascending=False)
l.head(20)

,VDEM
VDEM,1.000000
33.1 - No,0.527975
53,0.396248
40,0.383661
47.0 - No data,0.369609
"8.1 - No, bit specific limit",0.333390
"16.1 - Yes, for natural persons",0.317909
54.1,0.317564
31.3 - Previous election,0.302109
49,0.283276


In [16]:
# Implementation of feature selection method: variance treshold

X = variance_threshold_selector(X, .90 * (1 - .90))
# If line has over 85% of equal answers, we can exclude as it explains nothing

X

,1,3,5,7.1 - Yes,"7.2 - No, but specifc limit",7.3 - No,9,11,13.01 - Legal entities,13.02 -Other,...,62.5 - Court,62.7 - Auditing agency,62.8 - Ministry,\n63.1 - Other,63.4 - Loss of public funding,63.6 - Prison,63.7 - Deregistration of party,63.8 - Loss of nomination of candidate,63.9 - Loss of elected office,63.11 - Forfeiture
0,1,0,0,0,0,1,0,1,1,1,...,1,0,0,1,0,0,0,0,0,0
1,1,1,1,1,0,0,1,1,0,0,...,0,0,0,0,1,0,0,0,0,0
2,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,1,1,0,1,0,1,1,1,0,...,1,0,0,0,1,0,0,0,0,0
4,1,1,1,1,0,0,1,1,0,1,...,1,0,0,0,1,0,1,1,1,0
5,1,1,1,1,0,0,1,1,0,1,...,0,0,0,1,0,1,1,0,0,1
6,1,1,1,0,1,0,1,1,1,1,...,0,1,0,0,0,1,1,0,0,0
7,1,0,0,1,0,0,1,1,0,1,...,1,0,0,1,1,1,1,1,1,1
8,1,0,0,1,0,0,0,1,0,1,...,0,0,1,0,1,0,0,0,0,0
9,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,1,1,0,0,0,0


In [32]:
dici = {}
rsquared = []
questions = []
questionsCoefficients = []
questionsTtests = []
for c in X.columns:
    dvar = X[c]
    clf = regress(dvar,Y)
    #print(clf.summary())
    rsquared.append(clf.rsquared)
    questions.append(c)
    questionsCoefficients.append(dict(clf.params))
    questionsTtests.append(dict(clf.summary2().tables[1]['P>|t|']))

dici['Rsquared'] = rsquared
dici['Questions'] = questions
dici['Coefficients'] = questionsCoefficients
dici['P>|t|'] = questionsTtests
frame = pd.DataFrame.from_dict(dici)

junto = {}
dici2 = {}
questions = []
questionsCoefficients = []
questionsTtests = []
for e in dici['Coefficients']:
    junto.update(e)

for e in junto:
    questionsCoefficients.append(junto[e])
    questions.append(e)
    
junto = {}
for e in dici['P>|t|']:
    junto.update(e)

for e in junto:
    questionsTtests.append(junto[e])

    
dici2['Questions'] = questions
dici2['Coefficient'] = questionsCoefficients
dici2['P>|t|'] = questionsTtests
frame2 = pd.DataFrame.from_dict(dici2)


frame2 = frame2[frame2['P>|t|']<0.10]
frame2 = frame2.sort_values(by='Coefficient', ascending=False)

In [33]:
frame2 = frame2.head(20)
colunas = list(frame2['Questions'])
X = X[colunas]

In [75]:
colunas.remove('58.4 - Ministry')

In [76]:
#colunas.remove(38)
X = X[colunas]

In [77]:
results = regress(X,Y)
# Regression with selected features
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                   VDEM   R-squared (uncentered):                   0.898
Model:                            OLS   Adj. R-squared (uncentered):              0.874
Method:                 Least Squares   F-statistic:                              37.57
Date:                Wed, 22 Jul 2020   Prob (F-statistic):                    3.82e-13
Time:                        14:14:18   Log-Likelihood:                        -0.40257
No. Observations:                  37   AIC:                                      14.81
Df Residuals:                      30   BIC:                                      26.08
Df Model:                           7                                                  
Covariance Type:            nonrobust                                                  
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
59.7 - other                    0.3679      0.145      2.529      0.017       0.071       0.665
22.0 - No Data                  0.4259      0.146      2.921      0.007       0.128       0.724
62.8 - Ministry                 0.3064      0.127      2.409      0.022       0.047       0.566
7.2 - No, but specifc limit     0.2808      0.101      2.775      0.009       0.074       0.487
31.3 - Previous election        0.3954      0.082      4.819      0.000       0.228       0.563
29                              0.2683      0.094      2.851      0.008       0.076       0.460
13.05 - No                      0.3490      0.092      3.802      0.001       0.162       0.536
==============================================================================
Omnibus:                        0.371   Durbin-Watson:                   2.041
Prob(Omnibus):                  0.831   Jarque-Bera (JB):                0.534
Skew:                          -0.166   Prob(JB):                        0.766
Kurtosis:                       2.515   Cond. No.                         3.68
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
result = pd.merge(Y, X, left_index=True, right_index=True)
result = result.corr()
result = result.head(1)

plt.figure(figsize=(12,8))
sn.heatmap(result, square=False)
plt.show()